This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'APIKEY'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests as rq
from collections import namedtuple, defaultdict

from datetime import datetime

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
day_one_url = f"https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2017-01-01&end_date=2017-01-02&api_key={API_KEY}"
req = rq.get(day_one_url)
day_one_jsn = req.json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
day_one_jsn

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2020-06-26) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2020-07-10T22:16:32.795Z',
  'newest_available_date': '2020-07-10',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': None,
  'transform': None,
  'column_index': None,
  'start_date': '2017-01-01',
  'end_date': '2017-01-02',
  'data': [['2017-01-02',
    34.99,
    35.94,
    34.99,
    35.8,
    None,
    44700.0,
    1590561.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
year_url = f"https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2019-01-01&end_date=2019-12-31&api_key={API_KEY}"
req = rq.get(year_url)
year = req.json()

In [6]:
year_dict = defaultdict(list)
yrdct = {}
clmn = year['dataset']['column_names']
for dt in year['dataset']['data']:
    #dt[0] = datetime.strptime(dt[0], '%Y-%m-%d')
    a_day = tuple(zip(clmn, dt))
    for _key, val in a_day:
        #print(_key, val)
        #if _key not in yrdct:
        #    yrdct[_key] = []
        #yrdct[_key].append(val)
        year_dict[_key].append((val))
    # Let's calculate the largest change in a day
    year_dict['largest_change'].append(dt[2] - dt[3])
clean_open = sorted([i for i in year_dict['Open'] if i]) # gets rid of Nones and sort it
print(f'The highest opening price: {clean_open[-1]}')
print(f'The lowest opening price: {clean_open[0]}')

# Let's find the largest change in a day
largest_change = sorted([i for i in year_dict['largest_change'] if i])
print(f'The largest change in a day was: {largest_change[-1]}')

# The largest change between any two subsequent closings
Close = [i for i in year_dict['Close'] if i]
two_day_change = []
for i in range(len(Close)-1):
    two_day_change.append(abs(Close[i+1] - Close[i]))

two_day_change = sorted(two_day_change)
print(f'The largest change between any two subsequent closings: {two_day_change[-1]}')

# Average daily traded value
Traded_Vol = [i for i in year_dict['Traded Volume'] if i]
# we can't even use numpy since its not in standart lib. So we need to 
# do manual average'
k = 0
for val in Traded_Vol:
    k+=val
mean_trad_vol = k/(len(Traded_Vol))
print(f'The daily average of Traded Volums: {mean_trad_vol}')

# the median
Traded_Vol = sorted(Traded_Vol)
median_index = int(len(Traded_Vol)/2)
Traded_median = (Traded_Vol[median_index] + Traded_Vol[median_index +1])/2
print(f'The median of Traded Volums: {Traded_median}')

The highest opening price: 116.4
The lowest opening price: 83.8
The largest change in a day was: 11.450000000000003
The largest change between any two subsequent closings: 11.200000000000003
The daily average of Traded Volums: 158423.75454545455
The median of Traded Volums: 122785.5
